2D Slices extraction... refer the depths in kitsWorkspace_8 notebook

In [1]:
import os
import nibabel as nib
import numpy as np
import json

# Define the paths for the original images and masks
image_folder = "D:/kits23/resampled_and_sliced_images/"
mask_folder = "D:/kits23/resampled_and_sliced_segmentation_files/"
output_image_folder = "E:/kits23/2d_slices/images/"
output_mask_folder = "E:/kits23/2d_slices/masks/"
progress_file = "E:/kits23/processed_cases.json"  # JSON file to store progress

# Load the progress file (if exists)
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        processed_cases = json.load(f)
else:
    processed_cases = {}

# Create the output directories if they don't exist
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_mask_folder, exist_ok=True)

# Get the list of image files
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('windowed.nii.gz_resampled_and_sliced.nii.gz')]

# Generate the corresponding mask files based on the adjusted naming convention
mask_files = [
    os.path.join(mask_folder, f.replace('windowed.nii.gz_resampled_and_sliced.nii.gz', 'segmentation_resampled_and_sliced.nii.gz'))
    for f in os.listdir(image_folder) if f.endswith('windowed.nii.gz_resampled_and_sliced.nii.gz')
]

# Function to save 2D slices (images and masks) as .nii.gz files in case-specific folders
def save_slices(image_file, mask_file):
    # Extract the case number from the filename
    case_number = os.path.basename(image_file).split('_')[1]

    # Skip if this case has already been processed
    if case_number in processed_cases:
        print(f"Skipping case {case_number}, already processed.")
        return

    # Check if the mask file exists
    if not os.path.exists(mask_file):
        print(f"Warning: Mask file {mask_file} does not exist.")
        return

    # Load the 3D image and mask
    image = nib.load(image_file).get_fdata()
    mask = nib.load(mask_file).get_fdata()

    # Create directories for this case if they don't exist
    case_image_folder = os.path.join(output_image_folder, case_number)
    case_mask_folder = os.path.join(output_mask_folder, case_number)
    os.makedirs(case_image_folder, exist_ok=True)
    os.makedirs(case_mask_folder, exist_ok=True)

    # Iterate through each slice along the z-axis
    for i in range(image.shape[0]):
        slice_image = image[i, :, :]
        slice_mask = mask[i, :, :]

        # Create new 3D image objects with one slice (adding a new dimension)
        slice_image_3d = np.expand_dims(slice_image, axis=0)
        slice_mask_3d = np.expand_dims(slice_mask, axis=0)

        # Create the NIfTI image objects
        nifti_image = nib.Nifti1Image(slice_image_3d, affine=np.eye(4))
        nifti_mask = nib.Nifti1Image(slice_mask_3d, affine=np.eye(4))

        # Save the image and mask slices as .nii.gz files in the respective case folders
        slice_image_path = os.path.join(case_image_folder, f"slice_{i:03d}.nii.gz")
        slice_mask_path = os.path.join(case_mask_folder, f"slice_{i:03d}.nii.gz")

        nib.save(nifti_image, slice_image_path)
        nib.save(nifti_mask, slice_mask_path)

    # Print progress after processing all slices of a case
    print(f"Case {case_number} processing completed.")

    # Mark the case as processed
    processed_cases[case_number] = True

    # Save the progress to the file
    with open(progress_file, 'w') as f:
        json.dump(processed_cases, f)

# Process each image and mask pair
for image_file, mask_file in zip(image_files, mask_files):
    save_slices(image_file, mask_file)

print("2D image and mask slices extraction and saving completed.")


Skipping case 00000, already processed.
Skipping case 00001, already processed.
Skipping case 00002, already processed.
Skipping case 00003, already processed.
Skipping case 00004, already processed.
Skipping case 00005, already processed.
Skipping case 00006, already processed.
Skipping case 00007, already processed.
Skipping case 00008, already processed.
Skipping case 00009, already processed.
Skipping case 00010, already processed.
Skipping case 00011, already processed.
Skipping case 00012, already processed.
Skipping case 00013, already processed.
Skipping case 00014, already processed.
Skipping case 00015, already processed.
Skipping case 00016, already processed.
Skipping case 00017, already processed.
Skipping case 00018, already processed.
Skipping case 00019, already processed.
Skipping case 00020, already processed.
Skipping case 00021, already processed.
Skipping case 00022, already processed.
Skipping case 00023, already processed.
Skipping case 00024, already processed.


In [1]:
import os
import nibabel as nib
import numpy as np
import json
from PIL import Image

# Define the paths for the original NIfTI slices
nifti_image_folder = "E:/kits23/2d_slices/images/"
nifti_mask_folder = "E:/kits23/2d_slices/masks/"
output_png_image_folder = "E:/kits23/png_slices/images/"
output_png_mask_folder = "E:/kits23/png_slices/masks/"
progress_file = "E:/kits23/png_conversion_progress.json"  # JSON file to track progress

# Load progress from the file (if exists)
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        progress = json.load(f)
else:
    progress = {"images": {}, "masks": {}}

# Function to save NIfTI slices as PNG images
def save_slices_as_png(nifti_folder, output_folder, progress_key):
    for case in os.listdir(nifti_folder):
        case_path = os.path.join(nifti_folder, case)
        if not os.path.isdir(case_path):
            continue  # Skip files if any

        # Check if the output folder for this case already exists
        output_case_folder = os.path.join(output_folder, case)
        if os.path.exists(output_case_folder) and os.listdir(output_case_folder):
            print(f"Skipping case {case}, folder already exists.")
            continue  # Skip processing if the folder exists and contains files

        # Create the corresponding output case folder
        os.makedirs(output_case_folder, exist_ok=True)

        # Process each slice in the case folder
        for slice_file in os.listdir(case_path):
            if not slice_file.endswith('.nii.gz'):
                continue  # Skip non-NIfTI files

            # Load the NIfTI file
            nifti_file_path = os.path.join(case_path, slice_file)
            nifti_image = nib.load(nifti_file_path).get_fdata().squeeze()

            # Normalize the image to 0-255
            normalized_image = ((nifti_image - nifti_image.min()) / (nifti_image.max() - nifti_image.min()) * 255).astype(np.uint8)

            # Convert to PIL Image and save as PNG
            png_file_path = os.path.join(output_case_folder, slice_file.replace('.nii.gz', '.png'))
            Image.fromarray(normalized_image).save(png_file_path)

        print(f"Processed and saved PNG slices for case {case}.")

# Convert image slices to PNG
save_slices_as_png(nifti_image_folder, output_png_image_folder, "images")

# Convert mask slices to PNG
save_slices_as_png(nifti_mask_folder, output_png_mask_folder, "masks")

print("All NIfTI slices have been converted to PNG format.")


Skipping case 00000, folder already exists.
Skipping case 00001, folder already exists.
Skipping case 00002, folder already exists.
Skipping case 00003, folder already exists.
Skipping case 00004, folder already exists.
Skipping case 00005, folder already exists.
Skipping case 00006, folder already exists.
Skipping case 00007, folder already exists.
Skipping case 00008, folder already exists.
Skipping case 00009, folder already exists.
Skipping case 00010, folder already exists.
Skipping case 00011, folder already exists.
Skipping case 00012, folder already exists.
Skipping case 00013, folder already exists.
Skipping case 00014, folder already exists.
Skipping case 00015, folder already exists.
Skipping case 00016, folder already exists.
Skipping case 00017, folder already exists.
Skipping case 00018, folder already exists.
Skipping case 00019, folder already exists.
Skipping case 00020, folder already exists.
Skipping case 00021, folder already exists.
Skipping case 00022, folder alre

C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_16536\4084120891.py:47: RuntimeWarning: invalid value encountered in divide
  normalized_image = ((nifti_image - nifti_image.min()) / (nifti_image.max() - nifti_image.min()) * 255).astype(np.uint8)
C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_16536\4084120891.py:47: RuntimeWarning: invalid value encountered in cast
  normalized_image = ((nifti_image - nifti_image.min()) / (nifti_image.max() - nifti_image.min()) * 255).astype(np.uint8)


Processed and saved PNG slices for case 00038.
Processed and saved PNG slices for case 00039.
Processed and saved PNG slices for case 00040.
Processed and saved PNG slices for case 00041.
Processed and saved PNG slices for case 00042.
Processed and saved PNG slices for case 00043.
Processed and saved PNG slices for case 00044.
Processed and saved PNG slices for case 00045.
Processed and saved PNG slices for case 00046.
Processed and saved PNG slices for case 00047.
Processed and saved PNG slices for case 00048.
Processed and saved PNG slices for case 00049.
Processed and saved PNG slices for case 00050.
Processed and saved PNG slices for case 00051.
Processed and saved PNG slices for case 00052.
Processed and saved PNG slices for case 00053.
Processed and saved PNG slices for case 00054.
Processed and saved PNG slices for case 00055.
Processed and saved PNG slices for case 00056.
Processed and saved PNG slices for case 00057.
Processed and saved PNG slices for case 00058.
Processed and

In [4]:
import os
import nibabel as nib
import numpy as np
import json
from PIL import Image
import zipfile

# Define the paths for the original NIfTI slices
nifti_image_folder = "E:/kits23/2d_slices/images/"
nifti_mask_folder = "E:/kits23/2d_slices/masks/"
output_png_image_folder = "E:/kits23/png_slices/images/"
output_png_mask_folder = "E:/kits23/png_slices/masks/"
output_zip_image_folder = "E:/kits23/zip_slices/images/"
output_zip_mask_folder = "E:/kits23/zip_slices/masks/"
progress_file = "E:/kits23/png_conversion_progress.json"  # JSON file to track progress

# Load progress from the file (if exists)
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        progress = json.load(f)
else:
    progress = {"images": {}, "masks": {}}

# Create output directories if they don't exist
os.makedirs(output_zip_image_folder, exist_ok=True)
os.makedirs(output_zip_mask_folder, exist_ok=True)

# Function to save NIfTI slices as PNG images and create ZIP archives
def save_slices_and_zip(nifti_folder, output_folder, zip_folder, progress_key):
    for case in os.listdir(nifti_folder):
        case_path = os.path.join(nifti_folder, case)
        if not os.path.isdir(case_path):
            continue  # Skip files if any

        # Create the corresponding output case folder
        output_case_folder = os.path.join(output_folder, case)
        os.makedirs(output_case_folder, exist_ok=True)

        # Get processed slices for the case
        processed_slices = progress[progress_key].get(case, [])

        for slice_file in os.listdir(case_path):
            if not slice_file.endswith('.nii.gz') or slice_file in processed_slices:
                continue  # Skip non-NIfTI files or already processed slices

            # Load the NIfTI file
            nifti_file_path = os.path.join(case_path, slice_file)
            nifti_image = nib.load(nifti_file_path).get_fdata().squeeze()

            # Normalize the image to 0-255
            normalized_image = ((nifti_image - nifti_image.min()) / (nifti_image.max() - nifti_image.min()) * 255).astype(np.uint8)

            # Convert to PIL Image and save as PNG
            png_file_path = os.path.join(output_case_folder, slice_file.replace('.nii.gz', '.png'))
            Image.fromarray(normalized_image).save(png_file_path)

            # Update progress
            progress[progress_key].setdefault(case, []).append(slice_file)

            # Save progress to the file after processing each slice
            with open(progress_file, 'w') as f:
                json.dump(progress, f)

        # Create ZIP archive for the case folder
        zip_file_path = os.path.join(zip_folder, f"{case}.zip")
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(output_case_folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, output_case_folder)
                    zipf.write(file_path, arcname)

        print(f"Converted and archived all slices for case {case}.")

# Convert and archive image slices
save_slices_and_zip(nifti_image_folder, output_png_image_folder, output_zip_image_folder, "images")

# Convert and archive mask slices
save_slices_and_zip(nifti_mask_folder, output_png_mask_folder, output_zip_mask_folder, "masks")

print("All NIfTI slices have been converted to PNG format and archived.")


JSONDecodeError: Invalid control character at: line 1 column 131073 (char 131072)